### European up-and-in barrier call option for Wesfarmers Limited (WES)
**Information for purchase:**  
Strike Price = $80.00  
In-Barrier Level = $100.00  
Expires = 15th September 2027  
#### Step 1: Import Packages  


In [4]:
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib.lines import Line2D

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached scipy-1.15.3-cp313-cp313-macosx_14_0_arm64.whl.metadata (61 kB)
Using cached scipy-1.15.3-cp313-cp313-macosx_14_0_arm64.whl (22.4 MB)
Note: you may need to restart the kernel to use updated packages.
